<a href="https://colab.research.google.com/github/proteus21/DATA-SCIENCE-STUDY/blob/main/Machine%20Learning/08_association_rules/08_apriori_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### UCZENIE NIENADZOROWANE /  UNSUPERVISED LEARNING

# Association rules - apriori algorithm

A basic library for machine learning in Python
To install the scikit-learn library, use the command below:
```
!pip install scikit-learn
```
To update to the latest version of the scikit-learn library, use the command below:
```
!pip install --upgrade scikit-learn
```

### Contents:
1. [Import libraries](#0)
2. [Data generation ](#1)
3. [Data preaparing](#2)
4. [Transaction encoding](#3)
5. [Apriori algoritm](#4)




### <a name='0'></a> Import libraries

In [5]:
import pandas as pd
import numpy as np

### <a name='1'></a> Data generation

In [6]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
     

--2023-06-04 19:42:19--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.128, 142.250.141.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv.1’

products.csv.1      100%[===================>]   2.07M  2.40MB/s    in 0.9s    

2023-06-04 19:42:21 (2.40 MB/s) - ‘products.csv.1’ saved [2166953/2166953]

--2023-06-04 19:42:21--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.128, 142.250.141.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-

In [7]:
products=pd.read_csv('products.csv', usecols=['product_id','product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [8]:
orders=pd.read_csv('orders.csv', usecols=['order_id','product_id'])
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


### <a name='2'></a> Data preaparing

In [9]:
data=pd.merge(orders, products, how='inner',on='product_id', sort=True)
data=data.sort_values(by='order_id')
data

,order_id,product_id,product_name
588447,1,22035,Organic Whole String Cheese
256931,1,10246,Organic Celery Hearts
277441,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893,1,43633,Lightly Smoked Sardines in Olive Oil
1302365,1,47209,Organic Hass Avocado
...,...,...,...
981030,3421063,35548,Twice Baked Potatoes
350855,3421063,13565,No Salt Added Gluten-Free Blue Chips Made with...
101900,3421070,4724,Broccoli Florettes
993543,3421070,35951,Organic Unsweetened Almond Milk


In [10]:
data.describe()

,order_id,product_id
count,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04
std,9.897326e+05,1.412127e+04
min,1.000000e+00,1.000000e+00
25%,8.433700e+05,1.338000e+04
50%,1.701880e+06,2.529800e+04
75%,2.568023e+06,3.794000e+04
max,3.421070e+06,4.968800e+04


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 588447 to 433533
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   order_id      1384617 non-null  int64 
 1   product_id    1384617 non-null  int64 
 2   product_name  1384617 non-null  object
dtypes: int64(2), object(1)
memory usage: 42.3+ MB


In [12]:
#product distribuation
data['product_name'].value_counts()

Banana                                                               18726
Bag of Organic Bananas                                               15480
Organic Strawberries                                                 10894
Organic Baby Spinach                                                  9784
Large Lemon                                                           8135
                                                                     ...  
Original Raincoast Oat Crisp                                             1
Petite Creme Lowfat Strawberry Yogurt                                    1
Organic Petite Creme Mon Cherry Amour Yogurt                             1
Greek 100 Calories Whips! Peaches 'n Cream Fat Free Yogurt Mousse        1
Plain Golden Toasted Bread Crumbs                                        1
Name: product_name, Length: 39123, dtype: int64

In [13]:
#transaction number
data['order_id'].nunique()

131209

In [14]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda product: ','.join(product))
transactions

order_id
1          Organic Whole String Cheese,Organic Celery Hea...
36         Super Greens Salad,Spring Water,Organic Garnet...
38         Green Peas,Organic Baby Arugula,Shelled Pistac...
96         Organic Raspberries,Organic Pomegranate Kernel...
98         Uncured Applewood Smoked Bacon,Black Beans,Org...
                                 ...                        
3421049    Gluten Free Rice Bread,Organic Baby Broccoli,O...
3421056    Sparkling Lemon Water,Brioche Buns,Homestyle C...
3421058    Classic Britannia Crisps,Club Soda Lower Sodiu...
3421063    Natural Artesian Water,Organic Half & Half,Twi...
3421070    Broccoli Florettes,Organic Unsweetened Almond ...
Name: product_name, Length: 131209, dtype: object

In [15]:
transactions=transactions.str.split(',')
transactions

order_id
1          [Organic Whole String Cheese, Organic Celery H...
36         [Super Greens Salad, Spring Water, Organic Gar...
38         [Green Peas, Organic Baby Arugula, Shelled Pis...
96         [Organic Raspberries, Organic Pomegranate Kern...
98         [Uncured Applewood Smoked Bacon, Black Beans, ...
                                 ...                        
3421049    [Gluten Free Rice Bread, Organic Baby Broccoli...
3421056    [Sparkling Lemon Water, Brioche Buns, Homestyl...
3421058    [Classic Britannia Crisps, Club Soda Lower Sod...
3421063    [Natural Artesian Water, Organic Half & Half, ...
3421070    [Broccoli Florettes, Organic Unsweetened Almon...
Name: product_name, Length: 131209, dtype: object

### <a name='4'></a> Transaction encoding

In [16]:
from mlxtend.preprocessing import TransactionEncoder
encoder=TransactionEncoder()
encoder.fit(transactions)
transactions_encoded=encoder.transform( transactions, sparse=True)
transactions_encoded

<131209x40434 sparse matrix of type '<class 'numpy.bool_'>'
	with 1442410 stored elements in Compressed Sparse Row format>

In [17]:
transactions_encoded_df=pd.DataFrame(transactions_encoded.toarray(), columns=encoder.columns_)
transactions_encoded_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,...,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### <a name='5'></a> Apriori algorithm

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True, n_jobs=-1)
supports = supports.sort_values(by='support', ascending=False)
supports.head(10)

In [ ]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)